# Hyperparameter tuning

## Dask cluster

<table>
    <tr>
        <td>
            <img src="../_img/dask-horizontal.svg" width="300">
        </td>
        <td>
            <img src="../_img/snowflake.png" width="300">
        </td>
    </tr>
</table>

In [ ]:
import numpy as np

numeric_feat = [
    "pickup_weekday",
    "pickup_weekofyear",
    "pickup_hour",
    "pickup_week_hour",
    "pickup_minute",
    "passenger_count",
]
categorical_feat = [
    "pickup_taxizone_id",
    "dropoff_taxizone_id",
]
features = numeric_feat + categorical_feat
y_col = "tip_fraction"

# Initialize Dask Cluster

In [ ]:
import dask
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster(
    n_workers=n_workers,
    scheduler_size="medium",
    worker_size="large",
    nthreads=2,
)
client = Client(cluster)
cluster

Open the dashboard (link above ^) and watch it when you execute some commands, you'll see which tasks are running across the cluster.

If you created your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready.

>**Pro tip**: Create and/or start your cluster from the "Dask" page in Saturn if you want to get a head start!

In [ ]:
client.wait_for_workers(n_workers=n_workers)

# Load data and feature engineering

Load a sample from a single month for this exercise. Note we are loading the data with Dask now by defining a `dask.delayed` function to load partitions in parallel.

In [ ]:
import os
import dask.dataframe as dd

import snowflake.connector

SNOWFLAKE_ACCOUNT = os.environ["SNOWFLAKE_ACCOUNT"]
SNOWFLAKE_USER = os.environ["SNOWFLAKE_USER"]
SNOWFLAKE_PASSWORD = os.environ["SNOWFLAKE_PASSWORD"]

SNOWFLAKE_WAREHOUSE = os.environ["SNOWFLAKE_WAREHOUSE"]
TAXI_DATABASE = os.environ["TAXI_DATABASE"]
TAXI_SCHEMA = os.environ["TAXI_SCHEMA"]

conn_info = {
    "account": SNOWFLAKE_ACCOUNT,
    "user": SNOWFLAKE_USER,
    "password": SNOWFLAKE_PASSWORD,
    "warehouse": SNOWFLAKE_WAREHOUSE,
    "database": TAXI_DATABASE,
    "schema": TAXI_SCHEMA,
}
conn = snowflake.connector.connect(**conn_info)

The `load()` function specifies how each partition of the Dask dataframe should load its chunk of the data. As each partition of a Dask dataframe is a Pandas dataframe, we utilize the `fetch_pandas_all()` method.  We use a [binding for the Snowflake query](https://docs.snowflake.com/en/user-guide/python-connector-example.html#binding-data) so that we can pass different values at execution time. In this case, we pull one day of taxi rides into each partition.

In [ ]:
query = """
SELECT * FROM (
    SELECT
        pickup_taxizone_id,
        dropoff_taxizone_id,
        passenger_count,
        DIV0(tip_amount, fare_amount) AS tip_fraction,
        DAYOFWEEKISO(pickup_datetime) - 1 AS pickup_weekday,
        WEEKOFYEAR(pickup_datetime) AS pickup_weekofyear,
        HOUR(pickup_datetime) AS pickup_hour,
        (pickup_weekday * 24) + pickup_hour AS pickup_week_hour,
        MINUTE(pickup_datetime) AS pickup_minute
    FROM taxi_yellow
    WHERE
        DATE(pickup_datetime) = %s
) SAMPLE (1)
"""


@dask.delayed
def load(conn_info, query, day):
    with snowflake.connector.connect(**conn_info) as conn:
        taxi = conn.cursor().execute(query, str(day)).fetch_pandas_all()
        taxi.columns = taxi.columns.str.lower()
        return taxi

Run a query to determine which specific dates we need to pull

In [ ]:
def get_dates(start, end):
    date_query = """
    SELECT
        DISTINCT(DATE(pickup_datetime)) as date
    FROM taxi_yellow
    WHERE
        pickup_datetime BETWEEN %s and %s
    """
    dates_df = conn.cursor().execute(date_query, (start, end)).fetch_pandas_all()
    return dates_df["DATE"].tolist()


dates = get_dates("2019-01-01", "2019-01-31")

The `dd.from_delayed` function takes delayed Pandas outputs from a list and converts that to a high-level DataFrame object.

Since we're sampling in the Snowflake query, we run `persist()` to tell Dask to load all results into memory across the cluster. This ensures that Dask's lazy evaluation doesn't call the Snowflake query multiple times.

In [ ]:
taxi = dd.from_delayed([load(conn_info, query, day) for day in dates])
taxi = taxi.persist()
_ = wait(taxi)

In [ ]:
print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum().compute() / 1e6} MB")

In [ ]:
taxi_train = taxi[features + [y_col]].astype(float).fillna(-1)

In [ ]:
taxi_train.head()

# Run grid search

We use the preprocessing and `GridSearchCV` classes from dask-ml, but still use the scikit-learn `ElasticNet` model.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from dask_ml.compose import ColumnTransformer
from dask_ml.preprocessing import StandardScaler, DummyEncoder, Categorizer
from dask_ml.model_selection import GridSearchCV

pipeline = Pipeline(
    steps=[
        ("categorize", Categorizer(columns=categorical_feat)),
        ("onehot", DummyEncoder(columns=categorical_feat)),
        (
            "scale",
            ColumnTransformer(
                transformers=[("num", StandardScaler(), numeric_feat)],
                remainder="passthrough",
            ),
        ),
        ("clf", ElasticNet(normalize=False, max_iter=100)),
    ]
)

params = {
    "clf__l1_ratio": np.arange(0, 1.1, 0.1),
    "clf__alpha": [0, 0.5, 1, 2],
}

grid_search = GridSearchCV(pipeline, params, cv=3, scoring="neg_mean_squared_error")

Open up the Dask dashboard after you run the cell below, you'll see the grid search in action!

In [ ]:
%%time
_ = grid_search.fit(taxi_train[features], taxi_train[y_col])
grid_search.best_score_

In [ ]:
grid_search.best_params_

## Save model

`GridSearchCV` automatically fits the best parameters to the full data and stores in `best_estimator_`

In [ ]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/elastic_net_dask.pkl", "wb") as f:
    cloudpickle.dump(grid_search.best_estimator_, f)

## Calculate metrics on test set

Use a different month for test set

In [ ]:
test_dates = get_dates("2019-02-01", "2019-02-28")
taxi_test = dd.from_delayed([load(conn_info, query, day) for day in test_dates])
taxi_test = taxi_test.persist()
_ = wait(taxi_test)

In [ ]:
from sklearn.metrics import mean_squared_error

preds = grid_search.predict(taxi_test[features])
mean_squared_error(taxi_test[y_col], preds, squared=False)